##### 1. What is the estimated depth of a Decision Tree trained (unrestricted) on a one million instance training set ?

**Ans:** The estimated depth of a Decision Tree trained on a one million instance training set, without restrictions, can vary widely depending on factors like data complexity, features, and the tree-growing algorithm. However, it's typical for such a tree to have a depth ranging from dozens to hundreds or even more, potentially reflecting intricate decision boundaries. In highly complex datasets, the tree may need greater depth to accurately capture patterns. Nonetheless, with large datasets, maintaining interpretability and preventing overfitting remain challenges, often requiring pruning or ensemble techniques to manage depth effectively.







##### 2. Is the Gini impurity of a node usually lower or higher than that of its parent? Is it always lower/greater, or is it usually lower/greater ?

**Ans:** The Gini impurity of a node is typically lower than that of its parent. This reduction ensures the node's purity increases as the decision tree progresses, aiming to segregate classes better. However, it's not guaranteed to always decrease; in some cases, it may stay the same or increase marginally due to class distributions.







##### 3. Explain if its a good idea to reduce max depth if a Decision Tree is overfitting the training set ?

**Ans:** Reducing the max depth of a Decision Tree can help mitigate overfitting by limiting the complexity of the model, thus promoting generalization. It prevents the tree from growing too deep and capturing noise in the training data, leading to better performance on unseen data.







##### 4. Explain if its a  good idea to try scaling the input features if a Decision Tree underfits the training set ?

**Ans:** Scaling input features is generally unnecessary for decision trees as they're invariant to monotonic transformations. If a decision tree underfits, consider increasing its depth, reducing regularization, or using ensemble methods. Scaling might not significantly impact performance as decision trees inherently partition feature space without relying on absolute feature magnitudes.







##### 5. How much time will it take to train another Decision Tree on a training set of 10 million instances if it takes an hour to train a Decision Tree on a training set with 1 million instances ?

**Ans:** Training a Decision Tree on 10 million instances would likely take around 10 hours, given that it takes 1 hour to train on 1 million instances. The time complexity of training a Decision Tree typically scales linearly with the size of the training set, so the training time increases proportionally.







##### 6. Will setting presort=True speed up training if your training set has 100,000 instances ?

**Ans:** Setting presort=True might not speed up training significantly with 100,000 instances. Presorting requires sorting the data before each split, which could be time-consuming. It's more effective for smaller datasets or when memory constraints allow. For larger datasets, the overhead may outweigh the benefits.







##### 7. Follow these steps to train and fine-tune a Decision Tree for the moons dataset:
1. To build a moons dataset, use make moons(n samples=10000, noise=0.4).
2. Divide the dataset into a training and a test collection with train test split().
3. To find good hyperparameters values for a DecisionTreeClassifier, use grid search with cross-validation (with the GridSearchCV class). Try different values for max leaf nodes.
4. Use these hyperparameters to train the model on the entire training set, and then assess its output on the test set. You can achieve an accuracy of 85 to 87 percent.

In [3]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Step 1: Create moons dataset
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

# Step 2: Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Hyperparameter tuning using GridSearchCV
param_grid = {'max_leaf_nodes': [None, 10, 20, 30, 40, 50]}
dt_classifier = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dt_classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

# Step 4: Train model with best hyperparameters and evaluate on test set
best_dt_classifier = DecisionTreeClassifier(max_leaf_nodes=best_params['max_leaf_nodes'], random_state=42)
best_dt_classifier.fit(X_train, y_train)

test_accuracy = best_dt_classifier.score(X_test, y_test)
print("Test set accuracy:", test_accuracy)


Test set accuracy: 0.87


##### 8. Follow these steps to grow a forest:
1. Using the same method as before, create 1,000 subsets of the training set, each containing 100 instances chosen at random. You can do this with Scikit-ShuffleSplit Learn's class.
2. Using the best hyperparameter values found in the previous exercise, train one Decision Tree on each subset. On the test collection, evaluate these 1,000 Decision Trees. These Decision        Trees would likely perform worse than the first Decision Tree, achieving only around 80% accuracy, since they were trained on smaller sets.
3. Now the magic begins. Create 1,000 Decision Tree predictions for each test set case, and keep only the most common prediction (you can do this with SciPy's mode() function). Over the test collection, this method gives you majority-vote predictions.
4. On the test range, evaluate these predictions: you should achieve a slightly higher accuracy than the first model (approx 0.5 to 1.5 percent higher). You've successfully learned a Random Forest classifier!

In [4]:
from sklearn.datasets import make_moons
from sklearn.model_selection import ShuffleSplit
from sklearn.base import clone
import numpy as np

# Step 1: Create 1,000 subsets of the training set
n_trees = 1000
n_instances = 100

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
mini_sets = []

for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

# Step 2: Train one Decision Tree on each subset and evaluate on the test set
forest = [clone(grid_search.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    accuracy_scores.append(tree.score(X_test, y_test))

print("Average accuracy score of individual Decision Trees:", np.mean(accuracy_scores))

# Step 3: Perform majority vote predictions
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

# Step 4: Evaluate predictions
accuracy = np.mean(y_pred_majority_votes.reshape([-1]) == y_test)
print("Random Forest accuracy:", accuracy)


Average accuracy score of individual Decision Trees: 0.8012284999999999
Random Forest accuracy: 0.872
